## 1️⃣ استخدام في Node.js

In [ ]:
nodejs_basic = '''
// basic-usage.ts
import { Pose } from 'pose-format';
import * as fs from 'fs';

// قراءة ملف محلي
async function readLocalPose(filePath: string) {
    const pose = await Pose.fromLocal(filePath);
    
    console.log('=== معلومات الـ Pose ===');
    console.log(`Version: ${pose.header.version}`);
    console.log(`Width: ${pose.header.width}`);
    console.log(`Height: ${pose.header.height}`);
    console.log(`Depth: ${pose.header.depth}`);
    console.log(`FPS: ${pose.body.fps}`);
    console.log(`Frames: ${pose.body.frames.length}`);
    
    // المكونات
    console.log('\n=== المكونات ===');
    for (const comp of pose.header.components) {
        console.log(`- ${comp.name}: ${comp.points.length} points`);
    }
    
    return pose;
}

// تشغيل
readLocalPose('./sign_language.pose')
    .then(pose => console.log('تم التحميل بنجاح!'))
    .catch(err => console.error('خطأ:', err));
'''

print("📝 Node.js - الاستخدام الأساسي:")
print(nodejs_basic)

## 2️⃣ تحميل من URL في المتصفح

In [ ]:
browser_fetch = '''
// browser-fetch.ts
import { Pose } from 'pose-format';

class PoseLoader {
    private cache: Map<string, Pose> = new Map();
    
    async load(url: string, useCache = true): Promise<Pose> {
        // التحقق من الكاش
        if (useCache && this.cache.has(url)) {
            console.log('Loaded from cache:', url);
            return this.cache.get(url)!;
        }
        
        try {
            const pose = await Pose.fromRemote(url);
            
            if (useCache) {
                this.cache.set(url, pose);
            }
            
            return pose;
        } catch (error) {
            console.error('Failed to load pose:', error);
            throw error;
        }
    }
    
    async loadWithProgress(
        url: string, 
        onProgress?: (loaded: number, total: number) => void
    ): Promise<Pose> {
        const response = await fetch(url);
        
        if (!response.ok) {
            throw new Error(`HTTP error: ${response.status}`);
        }
        
        const contentLength = response.headers.get('content-length');
        const total = contentLength ? parseInt(contentLength, 10) : 0;
        let loaded = 0;
        
        const reader = response.body!.getReader();
        const chunks: Uint8Array[] = [];
        
        while (true) {
            const { done, value } = await reader.read();
            
            if (done) break;
            
            chunks.push(value);
            loaded += value.length;
            
            if (onProgress) {
                onProgress(loaded, total);
            }
        }
        
        // دمج الـ chunks
        const buffer = Buffer.concat(chunks.map(c => Buffer.from(c)));
        return Pose.from(buffer);
    }
    
    clearCache() {
        this.cache.clear();
    }
}

// استخدام
const loader = new PoseLoader();

// تحميل بسيط
loader.load('https://example.com/sign.pose')
    .then(pose => {
        console.log(`Loaded ${pose.body.frames.length} frames`);
    });

// تحميل مع شريط تقدم
loader.loadWithProgress(
    'https://example.com/sign.pose',
    (loaded, total) => {
        const percent = total > 0 ? (loaded / total * 100).toFixed(1) : 'N/A';
        console.log(`Loading: ${percent}%`);
    }
);
'''

print("📝 Browser - تحميل من URL:")
print(browser_fetch)

## 3️⃣ الوصول للبيانات

In [ ]:
data_access = '''
// data-access.ts
import { Pose, PoseHeaderModel, PoseBodyModel } from 'pose-format';

interface Point3D {
    X: number;
    Y: number;
    Z: number;
    C: number;  // Confidence
}

class PoseDataAccessor {
    constructor(private pose: Pose) {}
    
    // الحصول على عدد الإطارات
    get frameCount(): number {
        return this.pose.body.frames.length;
    }
    
    // الحصول على FPS
    get fps(): number {
        return this.pose.body.fps;
    }
    
    // الحصول على مدة الفيديو بالثواني
    get duration(): number {
        return this.frameCount / this.fps;
    }
    
    // الحصول على أسماء المكونات
    getComponentNames(): string[] {
        return this.pose.header.components.map(c => c.name);
    }
    
    // الحصول على أسماء النقاط في مكون
    getPointNames(componentName: string): string[] {
        const comp = this.pose.header.components.find(c => c.name === componentName);
        return comp?.points || [];
    }
    
    // الحصول على إطار كامل
    getFrame(frameIndex: number) {
        if (frameIndex < 0 || frameIndex >= this.frameCount) {
            throw new Error(`Frame index out of range: ${frameIndex}`);
        }
        return this.pose.body.frames[frameIndex];
    }
    
    // الحصول على نقطة محددة
    getPoint(
        frameIndex: number, 
        componentName: string, 
        pointIndex: number,
        personIndex: number = 0
    ): Point3D | null {
        const frame = this.getFrame(frameIndex);
        const person = frame.people[personIndex];
        
        if (!person) return null;
        
        const component = person[componentName];
        if (!component) return null;
        
        return component[pointIndex] as Point3D;
    }
    
    // الحصول على نقطة بالاسم
    getPointByName(
        frameIndex: number,
        componentName: string,
        pointName: string,
        personIndex: number = 0
    ): Point3D | null {
        const pointNames = this.getPointNames(componentName);
        const pointIndex = pointNames.indexOf(pointName);
        
        if (pointIndex === -1) return null;
        
        return this.getPoint(frameIndex, componentName, pointIndex, personIndex);
    }
    
    // الحصول على مسار نقطة عبر كل الإطارات
    getPointTrajectory(
        componentName: string,
        pointIndex: number,
        personIndex: number = 0
    ): Point3D[] {
        const trajectory: Point3D[] = [];
        
        for (let i = 0; i < this.frameCount; i++) {
            const point = this.getPoint(i, componentName, pointIndex, personIndex);
            if (point && point.C > 0) {
                trajectory.push(point);
            }
        }
        
        return trajectory;
    }
    
    // تحويل الإطار إلى وقت
    frameToTime(frameIndex: number): number {
        return frameIndex / this.fps;
    }
    
    // تحويل الوقت إلى إطار
    timeToFrame(timeSeconds: number): number {
        return Math.floor(timeSeconds * this.fps);
    }
}

// استخدام
async function example() {
    const pose = await Pose.fromRemote('https://example.com/sign.pose');
    const accessor = new PoseDataAccessor(pose);
    
    console.log(`Duration: ${accessor.duration.toFixed(2)} seconds`);
    console.log(`Components: ${accessor.getComponentNames().join(', ')}`);
    
    // الحصول على موقع الأنف في الإطار 0
    const nose = accessor.getPointByName(0, 'POSE_LANDMARKS', 'NOSE');
    if (nose) {
        console.log(`Nose at frame 0: X=${nose.X}, Y=${nose.Y}`);
    }
    
    // الحصول على مسار الرسغ الأيمن
    const rightWrist = accessor.getPointTrajectory('RIGHT_HAND_LANDMARKS', 0);
    console.log(`Right wrist trajectory: ${rightWrist.length} points`);
}
'''

print("📝 الوصول للبيانات:")
print(data_access)

## 4️⃣ رسم Pose على Canvas

In [ ]:
canvas_visualizer = '''
// canvas-visualizer.ts
import { Pose } from 'pose-format';

interface VisualizerOptions {
    pointRadius: number;
    lineWidth: number;
    pointColor: string;
    lineColor: string;
    backgroundColor: string;
    showConfidence: boolean;
}

const defaultOptions: VisualizerOptions = {
    pointRadius: 4,
    lineWidth: 2,
    pointColor: '#FF0000',
    lineColor: '#00FF00',
    backgroundColor: '#FFFFFF',
    showConfidence: false
};

class PoseCanvasVisualizer {
    private ctx: CanvasRenderingContext2D;
    private options: VisualizerOptions;
    private animationId: number | null = null;
    
    constructor(
        private canvas: HTMLCanvasElement,
        private pose: Pose,
        options: Partial<VisualizerOptions> = {}
    ) {
        this.ctx = canvas.getContext('2d')!;
        this.options = { ...defaultOptions, ...options };
        
        // ضبط حجم الـ canvas
        this.canvas.width = pose.header.width;
        this.canvas.height = pose.header.height;
    }
    
    // رسم إطار واحد
    drawFrame(frameIndex: number, personIndex: number = 0): void {
        const { ctx, pose, options } = this;
        
        // مسح الـ canvas
        ctx.fillStyle = options.backgroundColor;
        ctx.fillRect(0, 0, this.canvas.width, this.canvas.height);
        
        const frame = pose.body.frames[frameIndex];
        if (!frame || !frame.people[personIndex]) return;
        
        const person = frame.people[personIndex];
        
        // رسم كل مكون
        for (const component of pose.header.components) {
            const points = person[component.name];
            if (!points) continue;
            
            // رسم الخطوط (limbs)
            this.drawLimbs(points, component.limbs);
            
            // رسم النقاط
            this.drawPoints(points);
        }
    }
    
    private drawPoints(points: any[]): void {
        const { ctx, options } = this;
        
        for (const point of points) {
            if (point.C <= 0) continue;  // تخطي النقاط ذات الثقة صفر
            
            ctx.beginPath();
            ctx.arc(point.X, point.Y, options.pointRadius, 0, Math.PI * 2);
            
            if (options.showConfidence) {
                ctx.fillStyle = `rgba(255, 0, 0, ${point.C})`;
            } else {
                ctx.fillStyle = options.pointColor;
            }
            
            ctx.fill();
        }
    }
    
    private drawLimbs(points: any[], limbs: Array<{from: number, to: number}>): void {
        const { ctx, options } = this;
        
        ctx.strokeStyle = options.lineColor;
        ctx.lineWidth = options.lineWidth;
        
        for (const limb of limbs) {
            const p1 = points[limb.from];
            const p2 = points[limb.to];
            
            if (p1.C <= 0 || p2.C <= 0) continue;
            
            ctx.beginPath();
            ctx.moveTo(p1.X, p1.Y);
            ctx.lineTo(p2.X, p2.Y);
            ctx.stroke();
        }
    }
    
    // تشغيل الرسوم المتحركة
    play(onFrame?: (frameIndex: number) => void): void {
        let frameIndex = 0;
        const fps = this.pose.body.fps;
        const frameCount = this.pose.body.frames.length;
        const frameDuration = 1000 / fps;
        
        let lastTime = 0;
        
        const animate = (currentTime: number) => {
            if (currentTime - lastTime >= frameDuration) {
                this.drawFrame(frameIndex);
                
                if (onFrame) onFrame(frameIndex);
                
                frameIndex = (frameIndex + 1) % frameCount;
                lastTime = currentTime;
            }
            
            this.animationId = requestAnimationFrame(animate);
        };
        
        this.animationId = requestAnimationFrame(animate);
    }
    
    // إيقاف الرسوم المتحركة
    stop(): void {
        if (this.animationId !== null) {
            cancelAnimationFrame(this.animationId);
            this.animationId = null;
        }
    }
    
    // تصدير الإطار الحالي كصورة
    exportFrame(frameIndex: number): string {
        this.drawFrame(frameIndex);
        return this.canvas.toDataURL('image/png');
    }
}

// استخدام في صفحة HTML
/*
<canvas id="poseCanvas"></canvas>

<script>
async function init() {
    const canvas = document.getElementById('poseCanvas') as HTMLCanvasElement;
    const pose = await Pose.fromRemote('https://example.com/sign.pose');
    
    const visualizer = new PoseCanvasVisualizer(canvas, pose, {
        pointColor: '#FF5722',
        lineColor: '#2196F3',
        backgroundColor: '#FAFAFA'
    });
    
    // تشغيل
    visualizer.play((frame) => {
        console.log('Frame:', frame);
    });
    
    // إيقاف بعد 5 ثواني
    setTimeout(() => visualizer.stop(), 5000);
}

init();
</script>
*/
'''

print("📝 رسم على Canvas:")
print(canvas_visualizer)

## 5️⃣ مكون React للعرض

In [ ]:
react_component = '''
// PoseViewer.tsx
import React, { useEffect, useRef, useState } from 'react';
import { Pose } from 'pose-format';

interface PoseViewerProps {
    poseUrl?: string;
    poseData?: Pose;
    autoPlay?: boolean;
    loop?: boolean;
    showControls?: boolean;
    width?: number;
    height?: number;
    pointColor?: string;
    lineColor?: string;
    backgroundColor?: string;
}

export const PoseViewer: React.FC<PoseViewerProps> = ({
    poseUrl,
    poseData,
    autoPlay = true,
    loop = true,
    showControls = true,
    width,
    height,
    pointColor = '#FF0000',
    lineColor = '#00FF00',
    backgroundColor = '#FFFFFF'
}) => {
    const canvasRef = useRef<HTMLCanvasElement>(null);
    const [pose, setPose] = useState<Pose | null>(poseData || null);
    const [isPlaying, setIsPlaying] = useState(autoPlay);
    const [currentFrame, setCurrentFrame] = useState(0);
    const [loading, setLoading] = useState(false);
    const [error, setError] = useState<string | null>(null);
    const animationRef = useRef<number | null>(null);
    
    // تحميل الـ pose من URL
    useEffect(() => {
        if (poseUrl && !poseData) {
            setLoading(true);
            setError(null);
            
            Pose.fromRemote(poseUrl)
                .then(p => {
                    setPose(p);
                    setLoading(false);
                })
                .catch(err => {
                    setError(err.message);
                    setLoading(false);
                });
        }
    }, [poseUrl, poseData]);
    
    // رسم الإطار
    const drawFrame = (frameIndex: number) => {
        if (!pose || !canvasRef.current) return;
        
        const ctx = canvasRef.current.getContext('2d')!;
        const frame = pose.body.frames[frameIndex];
        
        // مسح
        ctx.fillStyle = backgroundColor;
        ctx.fillRect(0, 0, canvasRef.current.width, canvasRef.current.height);
        
        if (!frame || !frame.people[0]) return;
        
        const person = frame.people[0];
        
        for (const component of pose.header.components) {
            const points = person[component.name];
            if (!points) continue;
            
            // الخطوط
            ctx.strokeStyle = lineColor;
            ctx.lineWidth = 2;
            
            for (const limb of component.limbs) {
                const p1 = points[limb.from];
                const p2 = points[limb.to];
                
                if (p1?.C > 0 && p2?.C > 0) {
                    ctx.beginPath();
                    ctx.moveTo(p1.X, p1.Y);
                    ctx.lineTo(p2.X, p2.Y);
                    ctx.stroke();
                }
            }
            
            // النقاط
            ctx.fillStyle = pointColor;
            for (const point of points) {
                if (point?.C > 0) {
                    ctx.beginPath();
                    ctx.arc(point.X, point.Y, 4, 0, Math.PI * 2);
                    ctx.fill();
                }
            }
        }
    };
    
    // الرسوم المتحركة
    useEffect(() => {
        if (!pose || !isPlaying) return;
        
        const fps = pose.body.fps;
        const frameCount = pose.body.frames.length;
        const frameDuration = 1000 / fps;
        let lastTime = 0;
        let frame = currentFrame;
        
        const animate = (time: number) => {
            if (time - lastTime >= frameDuration) {
                drawFrame(frame);
                setCurrentFrame(frame);
                
                frame++;
                if (frame >= frameCount) {
                    if (loop) {
                        frame = 0;
                    } else {
                        setIsPlaying(false);
                        return;
                    }
                }
                
                lastTime = time;
            }
            
            animationRef.current = requestAnimationFrame(animate);
        };
        
        animationRef.current = requestAnimationFrame(animate);
        
        return () => {
            if (animationRef.current) {
                cancelAnimationFrame(animationRef.current);
            }
        };
    }, [pose, isPlaying, loop]);
    
    // ضبط حجم الـ canvas
    useEffect(() => {
        if (!canvasRef.current || !pose) return;
        canvasRef.current.width = width || pose.header.width;
        canvasRef.current.height = height || pose.header.height;
        drawFrame(currentFrame);
    }, [pose, width, height]);
    
    if (loading) return <div>Loading...</div>;
    if (error) return <div>Error: {error}</div>;
    if (!pose) return <div>No pose data</div>;
    
    return (
        <div className="pose-viewer">
            <canvas ref={canvasRef} />
            
            {showControls && (
                <div className="controls">
                    <button onClick={() => setIsPlaying(!isPlaying)}>
                        {isPlaying ? 'Pause' : 'Play'}
                    </button>
                    <span>
                        Frame: {currentFrame + 1} / {pose.body.frames.length}
                    </span>
                    <input
                        type="range"
                        min={0}
                        max={pose.body.frames.length - 1}
                        value={currentFrame}
                        onChange={(e) => {
                            const frame = parseInt(e.target.value);
                            setCurrentFrame(frame);
                            drawFrame(frame);
                        }}
                    />
                </div>
            )}
        </div>
    );
};

// استخدام
/*
<PoseViewer 
    poseUrl="https://example.com/sign.pose"
    autoPlay={true}
    loop={true}
    showControls={true}
    pointColor="#FF5722"
    lineColor="#2196F3"
/>
*/
'''

print("📝 مكون React:")
print(react_component)

## 6️⃣ تحليل الإشارات

In [ ]:
pose_analysis = '''
// pose-analysis.ts
import { Pose } from 'pose-format';

class PoseAnalyzer {
    constructor(private pose: Pose) {}
    
    // حساب المسافة بين نقطتين
    distance(p1: {X: number, Y: number}, p2: {X: number, Y: number}): number {
        return Math.sqrt((p2.X - p1.X) ** 2 + (p2.Y - p1.Y) ** 2);
    }
    
    // حساب سرعة الحركة لنقطة معينة
    calculateVelocity(
        componentName: string,
        pointIndex: number
    ): number[] {
        const velocities: number[] = [];
        const fps = this.pose.body.fps;
        
        for (let i = 1; i < this.pose.body.frames.length; i++) {
            const prev = this.pose.body.frames[i - 1].people[0]?.[componentName]?.[pointIndex];
            const curr = this.pose.body.frames[i].people[0]?.[componentName]?.[pointIndex];
            
            if (prev?.C > 0 && curr?.C > 0) {
                const dist = this.distance(prev, curr);
                velocities.push(dist * fps);  // pixels per second
            } else {
                velocities.push(0);
            }
        }
        
        return velocities;
    }
    
    // تحديد اللحظات التي تتحرك فيها اليد بسرعة
    findHighActivityFrames(threshold: number = 100): number[] {
        const leftVel = this.calculateVelocity('LEFT_HAND_LANDMARKS', 0);
        const rightVel = this.calculateVelocity('RIGHT_HAND_LANDMARKS', 0);
        
        const highActivity: number[] = [];
        
        for (let i = 0; i < leftVel.length; i++) {
            if (leftVel[i] > threshold || rightVel[i] > threshold) {
                highActivity.push(i + 1);  // +1 لأن velocity تبدأ من الإطار 1
            }
        }
        
        return highActivity;
    }
    
    // تحديد إذا كانت اليد مرفوعة
    isHandRaised(frameIndex: number, hand: 'left' | 'right'): boolean {
        const frame = this.pose.body.frames[frameIndex];
        const person = frame?.people[0];
        if (!person) return false;
        
        const handComp = hand === 'left' ? 'LEFT_HAND_LANDMARKS' : 'RIGHT_HAND_LANDMARKS';
        const shoulderPoint = hand === 'left' ? 'LEFT_SHOULDER' : 'RIGHT_SHOULDER';
        
        const wrist = person[handComp]?.[0];  // الرسغ عادة النقطة 0
        
        // البحث عن الكتف في POSE_LANDMARKS
        const posePoints = this.pose.header.components.find(c => c.name === 'POSE_LANDMARKS')?.points || [];
        const shoulderIdx = posePoints.indexOf(shoulderPoint);
        const shoulder = person['POSE_LANDMARKS']?.[shoulderIdx];
        
        if (!wrist || !shoulder || wrist.C <= 0 || shoulder.C <= 0) {
            return false;
        }
        
        // اليد مرفوعة إذا كانت أعلى من الكتف (Y أصغر)
        return wrist.Y < shoulder.Y;
    }
    
    // تقسيم الفيديو إلى مقاطع بناءً على النشاط
    segmentByActivity(minGapFrames: number = 10): Array<{start: number, end: number}> {
        const activity = this.findHighActivityFrames(50);
        if (activity.length === 0) return [];
        
        const segments: Array<{start: number, end: number}> = [];
        let segmentStart = activity[0];
        let lastFrame = activity[0];
        
        for (let i = 1; i < activity.length; i++) {
            if (activity[i] - lastFrame > minGapFrames) {
                segments.push({ start: segmentStart, end: lastFrame });
                segmentStart = activity[i];
            }
            lastFrame = activity[i];
        }
        
        segments.push({ start: segmentStart, end: lastFrame });
        
        return segments;
    }
    
    // إحصائيات عامة
    getStatistics() {
        const leftVel = this.calculateVelocity('LEFT_HAND_LANDMARKS', 0);
        const rightVel = this.calculateVelocity('RIGHT_HAND_LANDMARKS', 0);
        
        const avg = (arr: number[]) => arr.reduce((a, b) => a + b, 0) / arr.length;
        const max = (arr: number[]) => Math.max(...arr);
        
        return {
            duration: this.pose.body.frames.length / this.pose.body.fps,
            frameCount: this.pose.body.frames.length,
            fps: this.pose.body.fps,
            leftHand: {
                avgVelocity: avg(leftVel),
                maxVelocity: max(leftVel)
            },
            rightHand: {
                avgVelocity: avg(rightVel),
                maxVelocity: max(rightVel)
            },
            segments: this.segmentByActivity()
        };
    }
}

// استخدام
async function analyzeSign() {
    const pose = await Pose.fromRemote('https://example.com/sign.pose');
    const analyzer = new PoseAnalyzer(pose);
    
    const stats = analyzer.getStatistics();
    console.log('Statistics:', stats);
    
    // التحقق من رفع اليد في إطار معين
    const isRaised = analyzer.isHandRaised(50, 'right');
    console.log('Right hand raised at frame 50:', isRaised);
}
'''

print("📝 تحليل الإشارات:")
print(pose_analysis)

## 📚 ملخص

### التثبيت
```bash
npm install pose-format
```

### الاستخدام الأساسي
```typescript
import { Pose } from 'pose-format';

// Node.js
const pose = await Pose.fromLocal('./file.pose');

// Browser
const pose = await Pose.fromRemote('https://example.com/file.pose');

// من Buffer
const pose = Pose.from(buffer);
```

### الوصول للبيانات
```typescript
// معلومات عامة
pose.header.width
pose.header.height
pose.body.fps
pose.body.frames.length

// المكونات
pose.header.components

// نقطة محددة
const point = pose.body.frames[frameIndex].people[0].POSE_LANDMARKS[pointIndex];
```